In [16]:
import ipinfo
import pickle
import requests
import pandas as pd 
import numpy as np
from geopy.geocoders import Nominatim
import csv
import multiprocessing
import json
import pycountry
import country_converter as coco  #Coco was found to be more accuracte. Pycountry had weird labels for e.g. russia


dic = {
"Washington" : [
14166, 
15542, 
52915, 
54445, 
21003, 
12863, 
32616, 
53565, 
35776, 
55009, 
1001544, 
52574, 
1119, 
10331, 
21802, 
1001197, 
281, 
53097, 
53409, 
1002030, 
22675, 
1001199, 
51304, 
210231101, 
1002640, 
35498, 
4851, 
1004377, 
50142, 
1003891, 
21000, 
1001101, 
13520, 
6264, 
51694, 
6934, 
1000004, 
1000006, 
6947, 
54267, 
1000319, 
55298, 
54233, 
6407, 
1004250, 
1001507, 
34732, 
12681, 
32604, 
50468, 
12622, 
33798, 
55443, 
19911, 
10397,  
14242,  
54931,  
52883,  
12652,  
52883,  
12652,  
12611,  
53097,  
29287,  
25898,  
1058,
1004482,
10486,  
10743, 
16790, 
],
"Arizona" : [  
10905,  
1002766, 
28539, 
28592, 
53929, 
86, 
52326, 
51636, 
50787, 
1002591, 
31116, 
50936, 
1002928, 
6101, 
6408, 
1001552, 
1003541, 
1003531, 
1003533, 
1003535, 
1003537, 
1003538, 
1002799, 
1000732, 
6408, 
1001552, 
23000, 
1002539, 
1000296, 
33076, 
10456, 
12100, 
1002735, 
1002737, 
1002738, 
1004411, 
1001550, 
34770, 
1004540, 
30874, 
10394, 
55688, 
27798, 
19270, 
53929 
], 
"California" : [ 
#SOUTH CAL, I (not bcs it matter)
125, 
54136, 
12154, 
55681, 
1001796, 
26449, 
10185, 
32, 
1002526, 
6229, 
1001360, 
1001795, 
51293, 
28597, 
1001587, 
1002626, 
50379, 
52440, 
53492, 
21806, 
22447, 
591, 
51807, 
52419, 
55432, 
32083, 
51193, 
53927, 
55319, 
1004136, 
54896, 
16951, 
55050, 
30, 
1003489, 
16949, 
50620, 
31313, 
1003495, 
53965, 
54243, 
50338, 
15749, 
1004457, 
20184, 
50304, 
377, 
1000099, 
1164, 
54192, 
25860, 
1000011, 
60957, 
12116, 
1002655, 
15887, 
35056, 
2111, 
6811, 
1000112, 
1000109, 
6649, 
6989, 
19096, 
1003484, 
6492, 
32262, 
1004311, 
1003702, 
1000291, 
1002082, 
1004245, 
1002785, 
1004096, 
1003188, 
51053, 
6598, 
1002029, 
10556, 
54212, 
6831, 
55124, 
27761, 
27779, 
15958, 
53279, 
55330, 
16058, 
32567, 
# NORTH CALI 
16862, 
54121, 
12103, 
12426, 
1002532, 
53556, 
31169, 
12121, 
54294, 
1001600, 
27245, 
103, 
25156, 
52423, 
26772, 
27757, 
14203, 
55350, 
12999, 
14226, 
12710, 
24680, 
31423, 
13826, 
33897, 
12226, 
1000154, 
52417, 
33036, 
10403, 
13826, 
1004362, 
1002781, 
4111, 
14583, 
1003940, 
6992, 
6411, 
19383, 
345, 
13253, 
13561, 
13222, 
20431, 
10305, 
26147, 
55288, 
16785, 
6599, 
12671, 
55673, 
1002065, 
22379, 
11528, 
307, 
16641, 
22956, 
34517, 
54276, 
34787, 
1000673, 
7045, 
6898, 
1000748, 
1000119, 
50306, 
6836, 
30577, 
55514, 
54002, 
6833, 
34742, 
35093, 
19701, 
7121, 
6950, 
6480, 
1188, 
1004275, 
19956, 
14798, 
25393, 
28834, 
3669, 
1002312, 
1003892, 
1003711, 
33067, 
51113, 
34064, 
6782, 
1001181, 
23558, 
16056, 
52957, 
51615, 
6525, 
52618, 
1004143, 
19082, 
4307, 
55659, 
33492, 
34231, 
55173, 
32933, 
1004076, 
55112, 
50481, 
1001240, 
1000031, 
1002663, 
6685, 
1001467, 
1001458, 
1003797],
#### THEN , SKIPPED EVERYTHING ABOVE "SANFRANCISO BAY LABEL"
"Wyoming" : [
35140,
1004380,
55135
], 
"Texas" : [
35620, 
1003066, 
1002793, 
1000752, 
54743, 
18354, 
54937, 
54487, 
19963, 
34722, 
34631, 
1004406, 
33037, 
55754, 
52185, 
1003172, 
25029, 
54410, 
55311, 
51809, 
34084, 
1003937, 
34318, 
55635, 
52235, 
55536, 
32196, 
35594, 
10565, 
35771, 
13620, 
34691, 
35701, 
17739, 
10336, 
10344, 
34334, 
26735, 
52111, 
30303, 
1003157, 
50143, 
27214, 
52198, 
53101, 
50941, 
10317, 
1004493, 
53401, 
17914, 
50899, 
1001193, 
53923, 
1004458, 
52083, 
54539, 
50216, 
50456, 
32853, 
13694, 
18438, 
1198, 
12768, 
23017, 
1121, 
13283, 
12108, 
7038, 
50923, 
52383, 
55075, 
1001119, 
1003921, 
1001456, 
6409, 
1000515, 
1000523, 
6822, 
1002034, 
6938, 
4959, 
6988, 
1003680, 
1003583, 
1003164, 
1004077, 
13528, 
6692, 
64371002559, 
55741, 
1003066
],
"Illinois" : [
1002604, 
1001393, 
54251, 
55783, 
55712, 
19252, 
1004496, 
1001777, 
17226, 
25983, 
25988, 
1002366, 
25850, 
18425, 
25925, 
1004496, 
18741, 
12156, 
10545, 
54284, 
50223, 
19078, 
21905, 
55317, 
53206, 
12060, 
1003733, 
52634, 
10540, 
1002033, 
10194, 
14796, 
34220, 
19931, 
4085, 
16909, 
29004, 
1000105, 
2858, 
6280, 
6597, 
6987, 
32316, 
1002892, 
51388, 
50815, 
32471, 
12688, 
55655, 
52015, 
4019, 
11090, 
1004408, 
1004079, 
1004106, 
1004376, 
32555, 
53599, 
34287, 
1002093, 
54908, 
6629, 
6343
],
"Ontario" : [
55586, 
19522, 
15106, 
54109, 
55157, 
11876, 
52653, 
50549, 
50994, 
53148, 
15053, 
51762, 
34717, 
7070, 
1002303, 
33515, 
29379, 
15059, 
1000753, 
53079, 
1001823, 
17925, 
25001, 
1003907, 
31454, 
14099, 
20833, 
20628, 
26670, 
1002293, 
50315, 
53715, 
14255, 
1004000, 
53469, 
50189, 
1180, 
20615, 
54488, 
27120, 
53033, 
11792, 
13946, 
7001, 
6860, 
20447, 
19180, 
54882, 
24314, 
6924, 
27122, 
18095, 
10701, 
33116, 
12566, 
1001241, 
1004152, 
54525, 
1000026, 
55719, 
2100032, 
1001099, 
1000845, 
16595, 
34002, 
1003133, 
1003343, 
1004431, 
1004029, 
6456, 
6966, 
1003896, 
7088, 
3566, 
11792, 
53033, 
27120, 
13946, 
12124, 
25386, 
1004288, 
23085, 
32834, 
52606, 
12124, 
23016, 
29883, 
55308, 
14322, 
26201, 
53864, 
20519, 
51870, 
52828, 
17444, 
17408, 
33306, 
52166, 
1003692, 
15609, 
29594, 
1004489, 
33218, 
18896, 
17473, 
28533, 
14310, 
18022, 
30305, 
11209, 
28378, 
32341, 
33183, 
52473, 
33011, 
15156
],
"Quebec" : [ 
55710, 
52921, 
20186, 
35586, 
35634, 
20488, 
1003109, 
1003691, 
1002859, 
1002888, 
6719, 
1000849, 
1002089, 
55137, 
53294, 
22860, 
13244, 
54204, 
12895, 
54302, 
1002656, 
54399, 
17796, 
23157, 
19196, 
17922, 
32815, 
32976, 
14385, 
1004224, 
1004455, 
1003667, 
1004249, 
6710, 
19271, 
6462, 
53165, 
1002697, 
1003389, 
1003299, 
18203, 
14629, 
55657, 
10982, 
11506, 
55142, 
32815, 
32976, 
14385, 
52929, 
18306, 
23038, 
4599, 
6522, 
29183, 
51102, 
35230, 
31362, 
31436, 
55471, 
31207, 
18750, 
17304, 
19324, 
31302, 
24649, 
21567, 
1004372, 
31258, 
1000685, 
53016, 
23396
],
"Iowa" : [
6613, 
1002287, 
1003946, 
33080, 
1000968, 
50430, 
55078, 
1003145, 
35102, 
50128, 
1004418, 
55798, 
31492, 
51592, 
7119
],
"West-Virginia" : [
51343,
1000775,
24836,
21087
],
"Virginia" : [
#Westsouth to northeast
1002609, 
51101, 
51894, 
33673, 
1004581, 
1004193, 
50227, 
29138, 
34419, 
21700, 
1004369, 
33609, 
394, 
1004244, 
51631, 
30680, 
53981, 
53330, 
33646, 
10350, 
6428, 
10423, 
1004374, 
50455, 
32710, 
50220,  
28118,  
1004221, 
3574, 
54065, 
10445, 
14604, 
50017, 
20453, 
6797, 
10576, 
28194, 
12802, 
1001204, 
31389, 
15039, 
26934, 
1004119, 
1003053, 
1003051, 
34856, 
10746, 
1004295, 
1004310, 
1002709, 
24968, 
6289, 
1002707, 
1004246, 
6615, 
52434, 
1004144, 
1000174, 
1003385, 
1003902, 
1002633, 
1002272, 
10692, 
10746, 
1002690, 
4958, 
12506, 
15015, 
6524, 
6585, 
6479, 
1003184, 
6991, 
17194, 
50117, 
16071, 
54566, 
3737, 
1160, 
10462, 
1004212, 
55744, 
1003704, 
3410, 
1003674, 
54237, 
23749, 
13681, 
1002479, 
22881, 
6804, 
1002771, 
6928, 
6769, 
51331, 
31250, 
13249, 
15976, 
17423, 
11550, 
24725, 
23749, 
6341, 
50164, 
22382, 
54173, 
1004184, 
6588, 
50078, 
50476, 
10874, 
1000816, 
52449, 
17410, 
31330, 
1113, 
33415, 
52187, 
33415, 
1004187, 
1001204, 
31389, 
52227, 
50455, 
32710, 
1004374, 
6428, 
10350, 
33646, 
53330, 
53981, 
30680, 
51631, 
1004244, 
1004369
]} 


def get_ip():
    response = requests.get('https://api64.ipify.org?format=json').json()
    return response["ip"]


def get_location(ip):
    response = requests.get(f'https://ipapi.co/{ip}/json/').json()
    location_data = {
        "ip": ip,
        "city": response.get("city"),
        "region": response.get("region"),
        "country": response.get("country_name")
    }
    return location_data

def get_location_ipinfo(ip_address):
    try:
        access_token = '7dbb53d0419093'
        handler = ipinfo.getHandler(access_token)
        details = handler.getDetails(ip_address)
        if details.country == "US": 
            return details.region
        else:
            return details.country
    except:
        print("error")

@DeprecationWarning
def intersects_of_prbs_to_csv():

    df = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge.csv", index_col= False)
    dfs = [df[df["prb"].isin(dic[region])] for region in dic.keys()]
    for df in dfs: 
        for index, row in df.iterrows():
            df["MinIP_loc"] = get_location_ipinfo(row["MinIP"])
            for i in range(1,6):  
                name = f"otherIP{i}"
                if pd.isna(row[name]):
                    continue
                location_data = get_location_ipinfo(row[f"otherIP{i}"])
                df[f"otherIP{i}_loc"] = location_data
    concated = pd.concat([df for df in dfs], ignore_index=True)
    concated.to_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge_processed4.csv", index= False)

In [17]:
def add_ip_locations(df):
    df["MinIP_loc"] = np.nan
    df["otherIP1_loc"] = np.nan 
    df["otherIP2_loc"] = np.nan 
    df["otherIP3_loc"] = np.nan 
    df["otherIP4_loc"] = np.nan 
    df["otherIP5_loc"] = np.nan 

    for index, row in df.iterrows():
        df.loc[index, "MinIP_loc"] = get_location_ipinfo(row["MinIP"])
        for i in range(1,6):  
            name = f"otherIP{i}"
            if pd.isna(row[name]):
                continue

            location = get_location_ipinfo(row[f"otherIP{i}"])
            df.loc[index, f"otherIP{i}_loc"] = location
    return df

In [18]:
def create_adj_matrix(df):
    regions_set = set()
    cols = ["otherIP1_loc", "otherIP2_loc", "otherIP3_loc", "otherIP4_loc", "otherIP5_loc", "MinIP_loc"]
    for index, row in df.iterrows():
        for col in cols:
            regions_set.add(str(row[col]))
    regions_set.remove("None")
    regions_set.remove("nan")

    adj = pd.DataFrame(columns = list(regions_set))
    #39 STATES extracted

    for index, row in df.iterrows():
        for col in cols: 
            lat_col = col.replace("_loc", "")
            lat_col = lat_col.replace("IP", "latency")
            #try: 
            region_from = row[col]
            region_to = row["prb_loc"]
            region_from_idx = adj.columns.get_loc(region_from)
            region_to_idx = adj.columns.get_loc(region_to)
            
            latency = row.loc[lat_col]
            adj.iloc[region_from_idx, region_to_idx] = latency
            adj.iloc[region_to] = latency
            #except:
                #print("error")
    adj.to_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\adjance_latency.csv", index=False)
    return adj


In [19]:
def prb_location(df_prbs, prb_dic):
    geolocator = Nominatim(user_agent="geoapiExercises")
    df_prbs["prb_loc"] = ""
    for index, row in df_prbs.iterrows():
        prb_number = row["prb"]
        long,lat = tuple(prb_dic[prb_number]["geometry"]["coordinates"])
        location = ""
        try:
            location = geolocator.reverse(str(lat) + "," + str(long), language = 'en')
        except: 
            print("timeout at index: " + str(index))
            continue
        country = str((location)).split(",")[-1]
        # remove space from str
        country = country[1:]
        
        if country == "Canada":
            state = str((location)).split(",")[-3]
            # remove space from str
            state = state[1:]
            df_prbs.loc[index, "prb_loc"] = state
        elif country == "United States":
            state = str((location)).split(",")[-3]
            # remove space from str
            state = state[1:]
            df_prbs.loc[index, "prb_loc"] = state
        else: 
            df_prbs.loc[index, "prb_loc"] = country
        if index%1000 == 0:
            print("AT INDEX:   " + str(index))
    return df_prbs

In [20]:
def extract_prb_locs(df_edges):

    df_prbs_np_arr = df_edges["prb"].unique()
    df_prbs = pd.DataFrame(df_prbs_np_arr, columns=["prb"])

    with open(r'C:\Users\Admin\Documents\GitHub\umass\api\latencies\probes_clean.pickle', 'rb') as handle:
        prb_dic = pickle.load(handle)
        df_prb_loc = prb_location(df_prbs, prb_dic)
        df_prb_loc.to_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\prbs_with_locations.csv", index = False)
    return df_prbs


In [21]:
def merge_prb_locs_with_ip_locs(df_prb_loc, df_with_ip_locs):
    prb_to_loc = {}
    for index, row in df_prb_loc.iterrows():
        prb_to_loc[row["prb"]] = row["prb_loc"]
    
    df_with_ip_locs["prb_loc"] = ""
    for index, row in df_with_ip_locs.iterrows(): 
        df_with_ip_locs.loc[index,"prb_loc"] = prb_to_loc[row["prb"]]

    return df_with_ip_locs

def reorder_columns_of_final_df(merged_df):
    cols = [
        "prb" , "prb_loc",
        "MinIP", "MinIP_loc",
        "otherIP1", "otherIP1_loc", "otherlatency1",
        "otherIP2", "otherIP2_loc", "otherlatency2",
        "otherIP3", "otherIP3_loc", "otherlatency3",
        "otherIP4", "otherIP4_loc", "otherlatency4",
        "otherIP5", "otherIP5_loc", "otherlatency5",
        ]

    merged_df = merged_df[cols]
    
    merged_df = merged_df.rename({
        'otherIP1': 'IP1', 
        'otherIP2': 'IP2', 
        'otherIP3': 'IP3', 
        'otherIP4': 'IP4', 
        'otherIP5': 'IP5',
        'otherIP1_loc': 'IP1_loc', 
        'otherIP2_loc': 'IP2_loc', 
        'otherIP3_loc': 'IP3_loc', 
        'otherIP4_loc': 'IP4_loc', 
        'otherIP5_loc': 'IP5_loc',
        'otherlatency1': 'latency1', 
        'otherlatency2': 'latency2',
        'otherlatency3': 'latency3',
        'otherlatency4': 'latency4',
        'otherlatency5': 'latency5'
        }, axis='columns')

    return merged_df


In [22]:
#TAKES A COLUMN
def convert_country_code(df):
    
    pass


In [23]:
@DeprecationWarning
def prb_location_old(df_edges):
    geolocator = Nominatim(user_agent="geoapiExercises")
    df_edges["prb_loc"] = ""
    for index, row in df_edges.iterrows():
        prb_number = row["prb"]
        response = requests.get(f"https://atlas.ripe.net/api/v2/probes/{prb_number}/?format=json")
        resp_json = json.loads(response.content.decode("utf-8"))
        long, lat = tuple(resp_json["geometry"]["coordinates"])
        
        location = geolocator.reverse(str(lat) + "," + str(long), language = 'en')
        country = str((location)).split(",")[-1]
        
        if location == "United States":
            state = str((location)).split(",")[-3]
            df_edges.loc[index, "prb_loc"] = state
        else: 
            df_edges.loc[index, "prb_loc"] = country
    return df_edges

def singleprocess_prb_location():
    df_edges = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge.csv", index_col= False)
    df_prbs_np_arr = df_edges["prb"].unique()
    df_prbs = pd.DataFrame(df_prbs_np_arr, columns=["prb"])
    df_prbs = prb_location(df_prbs)
    df_prbs.to_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge_processed_testing2.csv", index = False)

@DeprecationWarning
def multiprocess_prb_location():
    df_edges = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge.csv", index_col= False)
    df_prbs_np_arr = df_edges["prb"].unique()
    df_prbs = pd.DataFrame(df_prbs_np_arr, columns=["prb"])
    num_processes = multiprocessing.cpu_count() - 1
    chunk_size = int(df_prbs.shape[0]/num_processes)
    chunks = [df_prbs.iloc[df_prbs.index[i:i + chunk_size]] for i in range(0, df_prbs.shape[0], chunk_size)]
    df_prbs = prb_location(df_prbs)

    pool = multiprocessing.Pool(processes=num_processes)
    result = pool.map(prb_location, chunks)

    for i in range(len(result)):
    # we can reassign the original dataframe based on the index of each chunk
        df_prbs.iloc[result[i].index] = result[i]

    df_prbs.to_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge_processed_testing2.csv", index = False)



In [26]:
def convert_country_code(df):
    cc = coco.CountryConverter()
    #df = df.apply(lambda x: pycountry.countries.get(name=str(x)) if pycountry.countries.get(name=str(x)) != None else print(x))
    df = df.apply(lambda x: coco.convert(names=x, to='ISO2') if coco.convert(names=x, to='ISO2', not_found = None) != None else x)
    return df

In [27]:
### This extracts probe id and maps it to a location

#df_edges = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge.csv", index_col= False)
#df_prb_locs = extract_prb_locs(df_edges)

### This assign a country code to non us/canada countries. they use states instead

df_prbs = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\prbs_with_locations.csv", index_col= False)
df_prbs["prb_loc"] = convert_country_code(df_prbs["prb_loc"])
df_prbs.to_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\prbs_with_locations_coco.csv", index = False)

### THIS adds location to the fields [MinIP, OtherIP{1,2,3,4,5}]

#df_with_ip_locs = add_ip_locations(df_edges)
#df_with_ip_locs.to_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge_with_ip_locs.csv", index= False)


### BELOW Is to merge files and formatting

# df_prb_loc = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\prbs_with_locations.csv", index_col = False)
# df_with_ip_locs = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge_processed10.csv", index_col = False)

# df_merged = merge_prb_locs_with_ip_locs(df_prb_loc, df_with_ip_locs)

# df_merged_formated = reorder_columns_of_final_df(df_merged)
# df_merged_formated.to_csv(r"C:\Users\Admin\Documents\GitHub\umass\api\latencies\edge_feat_locations.csv", index = False)
###



NameError: name 'coco' is not defined

In [ ]:
# geolocator = Nominatim(user_agent="geoapiExercises")

# long,lat = (-73.561668,45.508888)
    
# location = geolocator.reverse(str(lat) + "," + str(long), language = 'en')
# country = str((location)).split(",")[-1]
#         # remove space from str
# country = country[1:]
        
# if country == "Canada": 
#     print(location)